# Brainstat analysis comparing 3T and 7T epilepsy  

Surface-based comparisons  
- vertex-wise T-test : are controls and pts different
    - Use brainstat
- vertex-wise effect size : how big are the distances between the vertex differences?
    - Use own function


For figures: 
- Visualize effect size on a brain masked for significant p-values 

In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib
import brainstat as bstat

In [ ]:
# specify root directories
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_mp": "micapipe_v0.2.0",
    "dir_hu": "hippunfold_v1.3.0/hippunfold",
    "study": "3T"
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "study": "7T"
    }

studies = [MICs, PNI]

px_grps = { # specify patient group labels to compare to controls
    'all' : ['TLE_U', 'MFCL', 'FLE_R', 'MFCL_bTLE', 'UKN_L', 'mTLE_R', 'mTLE_L', 'FLE_L', 'UKN_U', 'TLE_L', 'TLE_R'],
    'TLE' : ['TLE_L', 'TLE_R', 'TLE_U', 'mTLE_R', 'mTLE_L'],
    'TLE_L': ['TLE_L', 'mTLE_L'],
    'TLE_R': ['TLE_R', 'mTLE_R'],
    'FCD' : ['FLE_R', 'FLE_L'],
    'MFCL' : ['MFCL', 'MFCL_bTLE'],
    'UKN' : ['UKN_L', 'UKN_U']
}

ctrl_grp = {'ctrl' : ['ctrl']}

surfaces = ["fsLR-5k"]
labels = ["pial", "white", "midThick"]

In [ ]:
# retrieve surfaces from pt of interest
def load_surf(study, sub_ses):
    """
    Get the surface data for a given group.

    inputs:
    study: dictionary item with keys 'name', 'dir_root', 'study'
    sub_ses: list of participant IDs and sessions to extract surfaces for

    outputs:
    concatenated surfaces in format accepted by BrainStats
    """
    
    import nibabel as nib
    
    # get the list of patients in the group
    pt_list = bstat.get_subjects(study, pt_grp)
    
    # get the surface data for each patient
    surf_data = {}
    for pt in pt_list:
        surf_data[pt] = {}
        for surf in surfaces:
            surf_data[pt][surf] = bstat.get_surface_data(study, pt, surf)
    
    return surf_data

In [ ]:
# when working, add to Utils scripts
def get_surfPath(root, deriv_fldr, sub, ses, label="midthickness", surf="fsLR-5k", space="nativepro", hemi="LR"):
    """
    Get the path to the surface data for a given subject and session.
    Assumes BIDS format of data storage.

    inputs:
        root: root directory of the study
        deriv_fldr: name of derivative folder containing the surface data
        sub: subject ID
        ses: session ID (with leading zero if applicable)
        surf: surface type and resolution (e.g., fsLR-32k, fsLR-5k)
        label: surface label (e.g., "pial", "white", "midThick")
        space: space of the surface data (e.g., "nativepro", "fsnative")
        hemi: hemisphere to extract (default is "LR" for both left and right hemispheres)

    outputs:
        path to the surface data files
    """

    # make surf to lower case
    label = label.lower()

    # ensure that surface is well defined
    if label == "pial":
        label = "pial"
    elif label == "white":
        label = "white"
    elif label == "midThick" or label == "midthickness":
        label = "midthickness"
    else:
        raise ValueError("Invalid surface type. Choose from 'pial', 'white', or 'midThick'.")
    
    # construct the path to the surface data file
    
    pth = f"{root}/derivatives/{deriv_fldr}/sub-{sub}/ses-{ses}/surf"
    
    hemi = hemi.upper()
    if hemi == "LEFT" or hemi == "L":
        hemi = "L"
    elif hemi == "RIGHT" or hemi == "R":
        hemi = "R"
    elif hemi != "LR":
        raise ValueError("Invalid hemisphere. Choose from 'L', 'R', or 'LR'.")

    # handle hippunfold naming convention
    if "hippunfold" in deriv_fldr.lower():
        # space usually: "T1w"
        # surf usually: "2mm"
        # label options: "hipp_outer", "hipp_inner", "hipp_midthickness"

        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_den-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            
            print(f"[surf_pth] Returning hippunfold paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_den-{surf}_label-{label}.surf.gii"

    elif "micapipe" in deriv_fldr.lower():
        if hemi == "LR":
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
        
            pth = [pth_L, pth_R]
            print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_R = f"{pth}/sub-{sub}_ses-{ses}_hemi-R_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth_L = f"{pth}/sub-{sub}_ses-{ses}_hemi-L_space-{space}_surf-{surf}_label-{label}.surf.gii"
            pth = [pth_L, pth_R]
            print(f"[surf_pth] Returning paths for both hemispheres ([0]: L, [1]: R)")
        elif hemi == "L" or hemi == "R":
            pth = f"{pth}/sub-{sub}_ses-{ses}_hemi-{hemi}_space-{space}_surf-{surf}_label-{label}.surf.gii"

    return pth


In [ ]:
# HANDLE MULTIPLE SESSIONS
# load surfaces by group

for study in studies:
    for grp in [ctrl_grp, px_grps]:
        # for each group, load surfaces into a single df
        # allows for comparisons between groups

        # add unique surfaces to a 3D-list. One such list per group. Indexes of the surface combination (hemi, surf, label, etc) is same across groups



In [ ]:
# flip TLEs --> put all lesions on same side

In [ ]:
# get difference maps at 3T (3T ctrl - 3T cases)

In [ ]:
# get difference maps at 7T (7T ctrl - 7T cases)

In [ ]:
# get difference maps of difference maps (3T dif maps - 7T dif maps)